In [1]:
#Import dependencies
import pandas as pd
import re

# Dependencies for MongoDB
import pymongo
from pprint import pprint  # For json output easy to read

# Dependency to convert dataframe to json format for MongoDB
import json

import requests
import os

import sys

from bs4 import BeautifulSoup as bs

In [2]:
url = "https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actress"
url

'https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actress'

In [3]:
res = requests.get(url)
res

<Response [200]>

In [4]:
soup = bs(res.text,'html.parser')
#soup

In [5]:
table = soup.find(class_= "wikitable sortable").find("tbody")
#table

In [6]:
year_data = table.find_all(scope = "row")
#year_data 

In [7]:
back_ground_yellow = table.find_all(style="background:#FAEB86;")
len(back_ground_yellow)

279

In [8]:
actress_list =[]
films_list = []
years_list = []
tie_index = []

for i in (range(len(year_data))):
    years_list.append(year_data[i].text.strip())  # total number of unique years are added to years_list

for i in (range(len(back_ground_yellow))):
    #print(i)
    if (i%3 == 0): # All first column elemnets in the table(actor names) are added to actors_list
        
        actor = back_ground_yellow[i].find_all("b")[0].find("span").find("a").text
        actress_list.append(actor)
     
        ind = int(i/3) # to get indexes for "year data" [0-92] so far
        # print(ind)
        
        if(back_ground_yellow[i].find("small")):  # if "Tie-up" winners are found, add indexes to "tie_index"
            #print(ind)
            #print("d")
            tie_index.append(ind)  
            if (len(tie_index)==2):  # Assuming 2 "Tie-up" winners in a year
                if (tie_index[-1]-1) == tie_index[-2]: # checking for last index and second last index
                    years_list.insert(tie_index[-1],years_list[tie_index[-2]])  #insert the "Year" value for the second "Tie-up" winner
                    tie_index.clear()  # Clear the list for the next scenario of 2 "Tie-up" winners 
                     

    if (i%3 == 2): # All third column elemnets in the table(Film names) are added to films_list
        
        films_list.append(back_ground_yellow[i].find("b").text)
 

In [9]:
len(actress_list)

93

In [10]:
len(films_list)

93

In [11]:
len(years_list)

93

In [12]:
bestactress_df = pd.DataFrame({"Years": years_list, "Winner Name":actress_list, "Film": films_list,"Category":"BestActress"})  
#bestactress_df.head(10)

In [13]:
actress_url = []
for i in actress_list:
    #print(i)
    actress_url.append((f"https://en.wikipedia.org/wiki/{i}"))

In [14]:
len(actress_url)

93

In [16]:
bestactress_df["Wiki URL"] = actress_url
bestactress_df.head(10)

,Years,Winner Name,Film,Category,Wiki URL
0,1927/28 (1st),Janet Gaynor,7th HeavenStreet AngelSunrise: A Song of Two H...,BestActress,https://en.wikipedia.org/wiki/Janet Gaynor
1,1928/29(2nd)[note 1],Mary Pickford,Coquette,BestActress,https://en.wikipedia.org/wiki/Mary Pickford
2,1929/30 (3rd),Norma Shearer,The Divorcee,BestActress,https://en.wikipedia.org/wiki/Norma Shearer
3,1930/31 (4th),Marie Dressler,Min and Bill,BestActress,https://en.wikipedia.org/wiki/Marie Dressler
4,1931/32 (5th),Helen Hayes,The Sin of Madelon Claudet,BestActress,https://en.wikipedia.org/wiki/Helen Hayes
5,1932/33 (6th),Katharine Hepburn,Morning Glory,BestActress,https://en.wikipedia.org/wiki/Katharine Hepburn
6,1934 (7th),Claudette Colbert,It Happened One Night,BestActress,https://en.wikipedia.org/wiki/Claudette Colbert
7,1935 (8th),Bette Davis,Dangerous,BestActress,https://en.wikipedia.org/wiki/Bette Davis
8,1936 (9th),Luise Rainer,The Great Ziegfeld,BestActress,https://en.wikipedia.org/wiki/Luise Rainer
9,1937 (10th),Luise Rainer,The Good Earth,BestActress,https://en.wikipedia.org/wiki/Luise Rainer


In [17]:
birthplace =[]
birthyear = []
succ = 0
fail = 0

for url in actress_url:
    response = requests.get(url)
    soup = bs(response.text,'html.parser')
 
    try:
        birthplace_result = soup.find_all('div', class_='birthplace')[0].text
        birthplace.append(birthplace_result)
    except: 
        print("Oops!",sys.exc_info()[0],"occured")
        birthplace.append('Unknown')
    try:
        birthyear_result = soup.find_all('span', class_='bday')[0].text[:4]
        birthyear.append(birthyear_result)
    except:
        print("Oops!",sys.exc_info()[0],"occured")
        birthyear.append('Unknown')
       
        

Oops! <class 'IndexError'> occured
Oops! <class 'IndexError'> occured
Oops! <class 'IndexError'> occured
Oops! <class 'IndexError'> occured


In [20]:
bestactress_df["Birthplace"] = birthplace
bestactress_df["Birth Year"] = birthyear

In [23]:
bestactress_df.head(10)

,Years,Winner Name,Film,Category,Wiki URL,Birthplace,Birth Year
0,1927/28 (1st),Janet Gaynor,7th HeavenStreet AngelSunrise: A Song of Two H...,BestActress,https://en.wikipedia.org/wiki/Janet Gaynor,"Germantown, Pennsylvania, U.S.",1906
1,1928/29(2nd)[note 1],Mary Pickford,Coquette,BestActress,https://en.wikipedia.org/wiki/Mary Pickford,"Toronto, Ontario, Canada",1892
2,1929/30 (3rd),Norma Shearer,The Divorcee,BestActress,https://en.wikipedia.org/wiki/Norma Shearer,"Montreal, Quebec, Canada",1902
3,1930/31 (4th),Marie Dressler,Min and Bill,BestActress,https://en.wikipedia.org/wiki/Marie Dressler,"Cobourg, Ontario, Canada",1868
4,1931/32 (5th),Helen Hayes,The Sin of Madelon Claudet,BestActress,https://en.wikipedia.org/wiki/Helen Hayes,"Washington, D.C., U.S.",1900
5,1932/33 (6th),Katharine Hepburn,Morning Glory,BestActress,https://en.wikipedia.org/wiki/Katharine Hepburn,"Hartford, Connecticut, U.S.",1907
6,1934 (7th),Claudette Colbert,It Happened One Night,BestActress,https://en.wikipedia.org/wiki/Claudette Colbert,"Saint-Mandé, France",1903
7,1935 (8th),Bette Davis,Dangerous,BestActress,https://en.wikipedia.org/wiki/Bette Davis,"Lowell, Massachusetts, U.S.",1908
8,1936 (9th),Luise Rainer,The Great Ziegfeld,BestActress,https://en.wikipedia.org/wiki/Luise Rainer,"Düsseldorf, Prussia, German Empire",1910
9,1937 (10th),Luise Rainer,The Good Earth,BestActress,https://en.wikipedia.org/wiki/Luise Rainer,"Düsseldorf, Prussia, German Empire",1910


In [19]:
len(birthplace)

93

In [69]:
len(birthyear)

93

In [82]:
#Import raw data from webpage
raw_df = pd.read_html("https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actress")[2]

In [83]:
#Create List of unique oscar years
list_of_years = raw_df.Year.unique()

In [84]:
#Loop through the data to select only the Best Actress winners in each year. 
#Best actress is always listed first in each Oscar year amongst the nominees.
years = []
actress=[]
film=[]

for x in list_of_years:
    years.append(raw_df[raw_df["Year"]==x].iloc[0]["Year"])
    actress.append(raw_df[raw_df["Year"]==x].iloc[0]["Actress"])
    film.append(raw_df[raw_df["Year"]==x].iloc[0]["Film"])

In [85]:
bestactress_df = pd.DataFrame({"Years": years, "Winner Name":actress, "Film": film,"Category":"BestActress"})

In [87]:
bestactress_df

,Years,Winner Name,Film,Category
0,1927/28 (1st),Janet Gaynor,7th HeavenStreet AngelSunrise: A Song of Two H...,BestActress
1,1928/29(2nd)[note 1],Mary Pickford,Coquette,BestActress
2,1929/30 (3rd),Norma Shearer [A],The Divorcee,BestActress
3,1930/31 (4th),Marie Dressler,Min and Bill,BestActress
4,1931/32 (5th),Helen Hayes,The Sin of Madelon Claudet,BestActress
...,...,...,...,...
87,2015 (88th),Brie Larson,Room,BestActress
88,2016 (89th),Emma Stone,La La Land,BestActress
89,2017 (90th),Frances McDormand,"Three Billboards Outside Ebbing, Missouri",BestActress
90,2018 (91st),Olivia Colman,The Favourite,BestActress


In [88]:
#Having trouble with Katherine Hepburn and Barbara Streisand tying in 1968. 
#Katherine Hepburn's name has (TIE) [C] after her name.
#They have special characters after their names and are also listed on separate lines.
#Norma Shearer has [A] after her name. I think these are endnotes. Removed.

In [89]:
actresses = pd.Series(actress).str.split("[").str[0].tolist()

In [90]:
actresses

['Janet Gaynor',
 'Mary Pickford',
 'Norma Shearer ',
 'Marie Dressler',
 'Helen Hayes',
 'Katharine Hepburn',
 'Claudette Colbert',
 'Bette Davis',
 'Luise Rainer',
 'Luise Rainer',
 'Bette Davis',
 'Vivien Leigh',
 'Ginger Rogers',
 'Joan Fontaine',
 'Greer Garson',
 'Jennifer Jones',
 'Ingrid Bergman',
 'Joan Crawford',
 'Olivia de Havilland',
 'Loretta Young',
 'Jane Wyman',
 'Olivia de Havilland',
 'Judy Holliday',
 'Vivien Leigh',
 'Shirley Booth',
 'Audrey Hepburn',
 'Grace Kelly',
 'Anna Magnani',
 'Ingrid Bergman',
 'Joanne Woodward',
 'Susan Hayward',
 'Simone Signoret',
 'Elizabeth Taylor',
 'Sophia Loren',
 'Anne Bancroft',
 'Patricia Neal',
 'Julie Andrews',
 'Julie Christie',
 'Elizabeth Taylor',
 'Katharine Hepburn',
 'Katharine Hepburn (TIE) ',
 'Maggie Smith',
 'Glenda Jackson',
 'Jane Fonda',
 'Liza Minnelli',
 'Glenda Jackson',
 'Ellen Burstyn',
 'Louise Fletcher',
 'Faye Dunaway',
 'Diane Keaton',
 'Jane Fonda',
 'Sally Field',
 'Sissy Spacek',
 'Katharine Hepburn',